In [3]:
# Cell 1: Setup
from pathlib import Path
from sec_risk import init_chroma
import pandas as pd

BASE = Path("./data")
PERSIST_DIR = str(BASE / "chroma_sec")
vectordb = init_chroma(PERSIST_DIR, collection_name="sec_10k_risk_factors")

print(f"✓ Database loaded: {vectordb._collection.count()} vectors")

✓ Database loaded: 12528 vectors


In [4]:
# Cell 2: Simple similarity search
query = "cybersecurity risks and data breaches"

results = vectordb.similarity_search_with_score(query, k=5)

print(f"Query: '{query}'")
print(f"Found {len(results)} results\n")

for i, (doc, score) in enumerate(results, 1):
    print(f"\n{'='*60}")
    print(f"Result {i} - Similarity Score: {score:.4f}")
    print(f"{'='*60}")
    print(f"Company: {doc.metadata['company']}")
    print(f"Form: {doc.metadata['form']}")
    print(f"Filing Date: {doc.metadata['filingDate']}")
    print(f"URL: {doc.metadata.get('url', 'N/A')}")
    print(f"\nContent preview:")
    print(doc.page_content[:400])
    print("...\n")

Query: 'cybersecurity risks and data breaches'
Found 5 results


Result 1 - Similarity Score: 0.3809
Company: HCA Healthcare, Inc.
Form: 10-K
Filing Date: 2023-02-17
URL: https://www.sec.gov/Archives/edgar/data/860730/000095017023003234/hca-20221231.htm

Content preview:
Risks related to technology, data privacy and cybersecurity:
 

A cybersecurity incident or other form of data breach could result in the compromise of our facilities, confidential data or critical data systems. A cybersecurity incident or other form of data breach could also give rise to potential harm to patients; remediation and other expenses; and exposure to liability under HIPAA, consumer pr
...


Result 2 - Similarity Score: 0.4296
Company: HCA Healthcare, Inc.
Form: 10-K
Filing Date: 2025-02-14
URL: https://www.sec.gov/Archives/edgar/data/860730/000095017025020134/hca-20241231.htm

Content preview:
not offer employment terms that are competitive with the rest of the labor market. Failure to attract, hire, devel

In [5]:
# Cell 3: Search with filters - specific company
company_name = "Apple Inc."  # Change to a company in your database

results = vectordb.similarity_search_with_score(
    query="supply chain risks",
    k=3,
    filter={"company": company_name}
)

print(f"Query: 'supply chain risks'")
print(f"Filter: Company = {company_name}")
print(f"Results: {len(results)}\n")

for i, (doc, score) in enumerate(results, 1):
    print(f"\nResult {i} - Score: {score:.4f}")
    print(f"Filing Date: {doc.metadata['filingDate']}")
    print(f"Content: {doc.page_content[:300]}...")

Query: 'supply chain risks'
Filter: Company = Apple Inc.
Results: 0



In [6]:
# Cell 4: Compare different queries
queries = [
    "cybersecurity threats",
    "climate change and environmental risks",
    "supply chain disruptions",
    "regulatory compliance",
    "competition and market risks"
]

results_summary = []

for query in queries:
    results = vectordb.similarity_search_with_score(query, k=3)
    
    for doc, score in results:
        results_summary.append({
            'query': query,
            'company': doc.metadata['company'],
            'score': score,
            'filing_date': doc.metadata['filingDate']
        })

df_results = pd.DataFrame(results_summary)
print("Top results by query:")
display(df_results.head(15))

# Best matches per query
print("\nBest match per query:")
best_matches = df_results.loc[df_results.groupby('query')['score'].idxmin()]
display(best_matches[['query', 'company', 'score']])

Top results by query:


,query,company,score,filing_date
0,cybersecurity threats,BERKSHIRE HATHAWAY INC,0.526822,2025-02-24
1,cybersecurity threats,Snowflake Inc.,0.539147,2025-03-21
2,cybersecurity threats,KKR & Co. Inc.,0.543640,2024-02-29
3,climate change and environmental risks,JOHNSON & JOHNSON,0.526495,2023-02-16
4,climate change and environmental risks,JOHNSON & JOHNSON,0.526495,2025-02-13
5,climate change and environmental risks,JOHNSON & JOHNSON,0.526495,2024-02-16
6,supply chain disruptions,COCA COLA CO,0.372037,2025-02-20
7,supply chain disruptions,COCA COLA CO,0.372037,2024-02-20
8,supply chain disruptions,COCA COLA CO,0.378674,2023-02-21
9,regulatory compliance,QUALCOMM INC/DE,0.629663,2024-11-06



Best match per query:


,query,company,score
3,climate change and environmental risks,JOHNSON & JOHNSON,0.526495
12,competition and market risks,AMGEN INC,0.437088
0,cybersecurity threats,BERKSHIRE HATHAWAY INC,0.526822
9,regulatory compliance,QUALCOMM INC/DE,0.629663
6,supply chain disruptions,COCA COLA CO,0.372037


In [7]:
# Cell 5: Interactive search function
def search_risks(query: str, n_results: int = 5, company: str = None):
    """
    Search for risk factors with optional company filter
    """
    filter_dict = {"company": company} if company else None
    
    results = vectordb.similarity_search_with_score(
        query=query,
        k=n_results,
        filter=filter_dict
    )
    
    print(f"🔍 Query: '{query}'")
    if company:
        print(f"🏢 Company: {company}")
    print(f"📊 Results: {len(results)}\n")
    
    for i, (doc, score) in enumerate(results, 1):
        print(f"\n{'─'*60}")
        print(f"#{i} | Score: {score:.4f} | {doc.metadata['company']}")
        print(f"📅 {doc.metadata['filingDate']} | {doc.metadata['form']}")
        print(f"{'─'*60}")
        print(doc.page_content[:400])
        print("...\n")
    
    return results

# Try it
results = search_risks("artificial intelligence and machine learning risks", n_results=3)

🔍 Query: 'artificial intelligence and machine learning risks'
📊 Results: 3


────────────────────────────────────────────────────────────
#1 | Score: 0.4509 | Uber Technologies, Inc
📅 2025-02-14 | 10-K
────────────────────────────────────────────────────────────
Our growing use of artificial intelligence and machine learning may present additional risks, including risks associated with algorithm development or use, the tools and data sets used, and/or a complex, developing regulatory environment.
...


────────────────────────────────────────────────────────────
#2 | Score: 0.4544 | Uber Technologies, Inc
📅 2024-02-15 | 10-K
────────────────────────────────────────────────────────────
Our growing use of artificial intelligence and machine learning may present additional risks, including risks associated with algorithm development or use, the data sets used, and/or a complex, developing regulatory environment.
...


────────────────────────────────────────────────────────────
#3 | Score

In [8]:
# Cell 6: Find similar documents to a given document
# Get a sample document
sample_doc = vectordb._collection.get(limit=1, include=['documents', 'metadatas'])

reference_text = sample_doc['documents'][0]
reference_company = sample_doc['metadatas'][0]['company']

print(f"Finding documents similar to:")
print(f"Company: {reference_company}")
print(f"Text preview: {reference_text[:200]}...\n")

# Search for similar content
similar = vectordb.similarity_search_with_score(reference_text, k=5)

print("Similar documents:")
for i, (doc, score) in enumerate(similar, 1):
    if doc.metadata['company'] != reference_company:  # Exclude same company
        print(f"{i}. {doc.metadata['company']} (Score: {score:.4f})")

Finding documents similar to:
Company: MICROSOFT CORP
Text preview: Item 1A...

Similar documents:
